In [1]:
# 操作 browser 的 API
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時的例外工具
from selenium.common.exceptions import TimeoutException

# 等待某個元素的出現
from selenium.webdriver.support.ui import WebDriverWait

# 期待元素出現並執行下一個指令
from selenium.webdriver.support import expected_conditions as EC

# 透過什麼方式選取元素
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# 強制等待
from time import sleep

# 反爬蟲
import undetected_chromedriver as uc

# 執行 command 的時候用的
import os

# 整理 json 使用的工具
import json

#計算程式執行的時間
import time

# 平行任務處理
from concurrent.futures import ThreadPoolExecutor as tpe

# regex
import re

# 檔案需轉成csv
import pandas as pd

from fake_useragent import UserAgent 
ua = UserAgent(cache= True)

my_options = uc.ChromeOptions()

my_options.add_argument("--start-maximized") # 視窗最大化
my_options.add_argument("--incognito") # 無痕
my_options.add_argument("--disable-popup-blocking") # 禁用彈出攔截
my_options.add_argument("--disable-notifiactions") # 關閉推波通知
my_options.add_argument("--lang=zh-TW") # 設定為繁體中文
my_options.add_argument("--headless")
# my_options.add_argument("--user-agent="+ua.random)

# 建立儲存檔案的資料夾
folderPath = "萬華區餐廳"
if os.path.exists(folderPath) != True:    
    os.makedirs(folderPath)

# 每搜尋一次計數器+1
index = 0

In [2]:
def page_link(link):
    
    global index
    
    # 自動取得Chrom driver 不用再額外取得 executable_path="chromedriver.exe"
    driver = uc.Chrome(options=my_options)
    
    # GoogleMap網址
    url = "https://www.google.com.tw/maps/@23.546162,120.6402133,8z?hl=zh-TW"
    
    driver.get(url)
    
    sleep(1)
    
    # 計算每跑一次跑多久
    start = time.time()
        
    # 輸入關鍵字搜尋
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).send_keys(link)
    
    # 按下搜尋按鈕
    driver.find_element(
        By.CSS_SELECTOR,
        "button#searchbox-searchbutton"
    ).click()
        
    sleep (3)
        
    # 找搜尋元素結果的div
    focus = driver.find_elements(
        By.CSS_SELECTOR,
        "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
    )[1]

    offset = 0
    innerHeight = 0
    count = 0  # 累計無效滾動次數
    limit = 5  # 最大無效滾動次數
    Flag = True
    counter = 0
        
    # 開始滾輪找到最底部
    while Flag:
            
        try:
            # 等待底部元素出現
            WebDriverWait(driver, 0.1).until(
                EC.presence_of_element_located( 
                    (By.CSS_SELECTOR, "span.HlvSq") 
                )
            )
            
            break
            
        except TimeoutException:

           # offset: 拉槓到頁面頂端的距離
            offset = driver.execute_script('return arguments[0].scrollTop', focus)

            # 執行js指令捲動頁面
            driver.execute_script('arguments[0].scrollTo(0, arguments[0].scrollHeight)', focus)
            
            # innerHeight: 頁面高度 = 拉槓到頁面頂端的距離
            innerHeight = driver.execute_script('return arguments[0].scrollHeight = arguments[0].scrollTop', focus)
            
            # 如果「拉槓到頁面頂端的距離」(offset)等於「頁面高度 = 拉槓到頁面頂端的距離」(innerHeight)，代表搜尋可能逾時
            if offset == innerHeight:
                count += 1                
              
            # 計數器等於限制數則搜尋確實逾時了,需重新整理並搜尋
            if count == limit:
                
                counter+=1
                
                print(f"{link}需重新整理,搜尋逾時,本次刷新第{counter}次")
                
                # 網頁重新整理
                driver.execute_script('window.location.replace(window.location.href)')
                
                # 重新搜尋
                driver.find_element(
                    By.CSS_SELECTOR,
                    "button#searchbox-searchbutton"
                 ).click()
                
                sleep(3)

                # 找搜尋結果的div
                focus = driver.find_elements(
                    By.CSS_SELECTOR,
                    "div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd"
                    )[1]

                # 重新定義需要的變數
                offset = 0
                innerHeight = 0
                count = 0  # 累計無效滾動次數
                
            sleep(1)
            
    # 尋找所有元素的href    
    href_ = driver.find_elements(
        By.CSS_SELECTOR,
        "div[data-js-log-root] a[aria-label]"
    )
    
    
    # 放xx區裡的xx路餐廳的網址lict
    listUrl=[]
    counter = 0 

    # 把所有href加進listUrl裡
    for href in href_:        
        listUrl.append({
       
            "href":href.get_attribute("href")
        })      
        counter+=1
    
    # 把餐廳網址list先轉成dataframe再存成csv        
    df = pd.DataFrame(listUrl)
    df.to_csv(f'{folderPath}/{folderPath}.csv',mode='a', header=False, index=False)
    
    # 清除搜尋的關鍵字
    driver.find_element(
        By.CSS_SELECTOR,
        "input#searchboxinput"
    ).clear()   
    
    # 每搜尋成功並寫入就計數器+1
    index+=1
    
    end = time.time()
    print(f'第{index}筆 {link},總共有:{counter}筆,已寫入完成')
    print("process_time 測量時間：%f 秒" % (end - start))
    
    driver.quit()

    # 每條(街or路)的全部restaurantType都抓完,則提示已完成全(街or路)抓取,,總共restaurantType有26種
    if index%26 == 0:
        regex01 = r'.+(街|路)'
        string01 = link
        match01 = re.search(regex01, string01)
        print(f'注意!!!{match01[0]}:已全數寫入完成\n')
    else:
        print("繼續抓\n")
        

In [3]:
def multiprocess():
    
    district = "萬華區" 
    
    roadNames = ["三水街","中華路１段","中華路２段","內江街","南寧路","和平西路２段","和平西路３段","國興路","大埔街","大理街","富民路","寶興街","峨眉街","康定路","廣州街","德昌街","忠孝西路２段","成都路","昆明街","東園街","柳州街","桂林路","梧州街","武成街","武昌街２段","民和街","水源路","永福街","汀州路１段","洛陽街","漢中街","漢口街２段","環河南路１段","環河南路２段","環河南路３段","興寧街","興義街","艋舺大道","莒光路","華西街","萬大路","萬青街","西園路１段","西園路２段","西寧南路","西昌街","西藏路","貴陽街２段","長沙街２段","長泰街","長順街","開封街２段","隆昌街","雅江街","雙和街","雙園街","青年路"]
        
    # 26種
    restaurantType = ["火鍋","拉麵","日式","美式","義式","法式","中式","台式","韓式","德式","地中海料理","印度料理","越式","港式","泰式","南洋","素食","鐵板燒","餐酒館","咖啡廳","熱炒店","早午餐","甜點店","燒肉","海鮮餐廳","牛排"]

    drt = []
    
    for road in roadNames:
        for type_ in restaurantType:
            keyword = f'{district}{road}{type_}' 
            # 平行運算x2,
            drt.append(keyword)
    
    with tpe(max_workers=5) as executor:
        executor.map(page_link,drt) # 執行page_link涵式,drt是涵式需要的變數
      

In [4]:
if __name__=="__main__":
    multiprocess()

第1筆 萬華區三水街日式,總共有:43筆,已寫入完成
process_time 測量時間：24.041994 秒

繼續抓

萬華區三水街火鍋需重新整理,搜尋逾時,本次刷新第1次
萬華區三水街美式需重新整理,搜尋逾時,本次刷新第1次
第2筆 萬華區三水街義式,總共有:120筆,已寫入完成
process_time 測量時間：47.877480 秒

第3筆 萬華區三水街拉麵,總共有:120筆,已寫入完成
process_time 測量時間：49.221271 秒

繼續抓

繼續抓

第4筆 萬華區三水街德式,總共有:39筆,已寫入完成
process_time 測量時間：8.470945 秒

繼續抓

第5筆 萬華區三水街法式,總共有:120筆,已寫入完成
process_time 測量時間：40.870686 秒

第6筆 萬華區三水街美式,總共有:120筆,已寫入完成
process_time 測量時間：76.781424 秒

繼續抓

繼續抓

第7筆 萬華區三水街韓式,總共有:120筆,已寫入完成
process_time 測量時間：40.789622 秒

繼續抓

第8筆 萬華區三水街地中海料理,總共有:120筆,已寫入完成
process_time 測量時間：40.951577 秒

繼續抓

第9筆 萬華區三水街印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.463190 秒

繼續抓

萬華區三水街港式需重新整理,搜尋逾時,本次刷新第1次
第10筆 萬華區三水街越式,總共有:120筆,已寫入完成
process_time 測量時間：40.192750 秒

繼續抓

萬華區三水街南洋需重新整理,搜尋逾時,本次刷新第1次
萬華區三水街泰式需重新整理,搜尋逾時,本次刷新第1次
第11筆 萬華區三水街素食,總共有:120筆,已寫入完成
process_time 測量時間：40.559234 秒

第12筆 萬華區三水街港式,總共有:120筆,已寫入完成
process_time 測量時間：70.868428 秒

繼續抓

萬華區三水街鐵板燒需重新整理,搜尋逾時,本次刷新第1次
繼續抓

第13筆 萬華區三水街南洋,總共有:195筆,已寫入完成
process_time 測量時間：70.915798 秒

繼續抓


第9筆 萬華區南寧路德式,總共有:120筆,已寫入完成
process_time 測量時間：90.238588 秒

繼續抓

萬華區中華路２段甜點店需重新整理,搜尋逾時,本次刷新第16次
第10筆 萬華區南寧路印度料理,總共有:120筆,已寫入完成
process_time 測量時間：84.581022 秒

繼續抓

第11筆 萬華區南寧路地中海料理,總共有:120筆,已寫入完成
process_time 測量時間：88.611516 秒

繼續抓

萬華區南寧路泰式需重新整理,搜尋逾時,本次刷新第1次
萬華區南寧路南洋需重新整理,搜尋逾時,本次刷新第1次
萬華區中華路２段甜點店需重新整理,搜尋逾時,本次刷新第17次
萬華區南寧路素食需重新整理,搜尋逾時,本次刷新第1次
萬華區南寧路餐酒館需重新整理,搜尋逾時,本次刷新第1次
萬華區南寧路甜點店需重新整理,搜尋逾時,本次刷新第1次
萬華區中華路２段甜點店需重新整理,搜尋逾時,本次刷新第18次
第12筆 萬華區南寧路早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.962343 秒

第13筆 萬華區南寧路泰式,總共有:120筆,已寫入完成
process_time 測量時間：104.451781 秒

繼續抓

繼續抓

第14筆 萬華區南寧路餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：66.407258 秒

繼續抓

萬華區中華路２段甜點店需重新整理,搜尋逾時,本次刷新第19次
萬華區南寧路甜點店需重新整理,搜尋逾時,本次刷新第2次
萬華區南寧路海鮮餐廳需重新整理,搜尋逾時,本次刷新第1次
萬華區和平西路２段火鍋需重新整理,搜尋逾時,本次刷新第1次
萬華區中華路２段甜點店需重新整理,搜尋逾時,本次刷新第20次
萬華區南寧路甜點店需重新整理,搜尋逾時,本次刷新第3次
第15筆 萬華區南寧路海鮮餐廳,總共有:79筆,已寫入完成
process_time 測量時間：64.837123 秒

繼續抓

第16筆 萬華區和平西路２段拉麵,總共有:120筆,已寫入完成
process_time 測量時間：49.709078 秒

第17筆 萬華區和平西路２段火鍋,總共有:111筆,已寫入完成
process_time 測量時間：51.289651 秒

繼續抓


繼續抓

萬華區大理街韓式需重新整理,搜尋逾時,本次刷新第1次
第17筆 萬華區大理街德式,總共有:148筆,已寫入完成
process_time 測量時間：24.994479 秒

繼續抓

第18筆 萬華區大理街地中海料理,總共有:120筆,已寫入完成
process_time 測量時間：40.505924 秒

繼續抓

第19筆 萬華區大理街印度料理,總共有:120筆,已寫入完成
process_time 測量時間：40.406939 秒

繼續抓

第20筆 萬華區大理街越式,總共有:120筆,已寫入完成
process_time 測量時間：40.807495 秒

繼續抓

第21筆 萬華區大理街韓式,總共有:121筆,已寫入完成
process_time 測量時間：84.874891 秒

繼續抓

第22筆 萬華區大理街港式,總共有:120筆,已寫入完成
process_time 測量時間：40.568575 秒

繼續抓

萬華區大理街泰式需重新整理,搜尋逾時,本次刷新第1次
萬華區大理街素食需重新整理,搜尋逾時,本次刷新第1次
第23筆 萬華區大理街南洋,總共有:120筆,已寫入完成
process_time 測量時間：40.673478 秒

萬華區大理街鐵板燒需重新整理,搜尋逾時,本次刷新第1次
繼續抓

萬華區大理街咖啡廳需重新整理,搜尋逾時,本次刷新第1次
第24筆 萬華區大理街餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：40.873205 秒

繼續抓

第25筆 萬華區大理街素食,總共有:120筆,已寫入完成
process_time 測量時間：72.110584 秒

繼續抓

第26筆 萬華區大理街熱炒店,總共有:120筆,已寫入完成
process_time 測量時間：40.843741 秒

第27筆 萬華區大理街鐵板燒,總共有:120筆,已寫入完成
process_time 測量時間：72.470358 秒

繼續抓

繼續抓

第28筆 萬華區大理街甜點店,總共有:14筆,已寫入完成
process_time 測量時間：6.422216 秒

繼續抓

第29筆 萬華區大理街咖啡廳,總共有:120筆,已寫入完成
process_time 測量時間：70.466178 秒

繼續抓

第30筆 

繼續抓

萬華區廣州街餐酒館需重新整理,搜尋逾時,本次刷新第1次
萬華區廣州街鐵板燒需重新整理,搜尋逾時,本次刷新第1次
第23筆 萬華區廣州街素食,總共有:120筆,已寫入完成
process_time 測量時間：40.744721 秒

萬華區廣州街咖啡廳需重新整理,搜尋逾時,本次刷新第1次
繼續抓

萬華區廣州街甜點店需重新整理,搜尋逾時,本次刷新第1次
萬華區廣州街熱炒店需重新整理,搜尋逾時,本次刷新第1次
第24筆 萬華區廣州街早午餐,總共有:120筆,已寫入完成
process_time 測量時間：44.515128 秒

繼續抓

第25筆 萬華區廣州街燒肉,總共有:120筆,已寫入完成
process_time 測量時間：40.468700 秒

第26筆 萬華區廣州街海鮮餐廳,總共有:120筆,已寫入完成
process_time 測量時間：40.659752 秒

注意!!!萬華區廣州街:已全數寫入完成

注意!!!萬華區廣州街:已全數寫入完成

第1筆 萬華區廣州街牛排,總共有:119筆,已寫入完成
process_time 測量時間：40.535951 秒

繼續抓

萬華區德昌街火鍋需重新整理,搜尋逾時,本次刷新第1次
第2筆 萬華區廣州街熱炒店,總共有:120筆,已寫入完成
process_time 測量時間：75.867951 秒

繼續抓

萬華區德昌街美式需重新整理,搜尋逾時,本次刷新第1次
第3筆 萬華區德昌街拉麵,總共有:120筆,已寫入完成
process_time 測量時間：40.354135 秒

第4筆 萬華區德昌街日式,總共有:120筆,已寫入完成
process_time 測量時間：40.455636 秒

繼續抓

繼續抓

萬華區德昌街火鍋需重新整理,搜尋逾時,本次刷新第2次
第5筆 萬華區德昌街義式,總共有:120筆,已寫入完成
process_time 測量時間：40.418451 秒

繼續抓

第6筆 萬華區德昌街美式,總共有:120筆,已寫入完成
process_time 測量時間：62.282813 秒

繼續抓

第7筆 萬華區德昌街法式,總共有:120筆,已寫入完成
process_time 測量時間：40.541622 秒

萬華區德昌街火鍋需重新整理,搜尋逾時,本次刷新第3次
繼續抓

萬華區

第23筆 萬華區東園街鐵板燒,總共有:107筆,已寫入完成
process_time 測量時間：37.186736 秒

繼續抓

萬華區東園街燒肉需重新整理,搜尋逾時,本次刷新第1次
第24筆 萬華區東園街熱炒店,總共有:103筆,已寫入完成
process_time 測量時間：35.828262 秒

萬華區東園街甜點店需重新整理,搜尋逾時,本次刷新第1次
繼續抓

第25筆 萬華區東園街早午餐,總共有:120筆,已寫入完成
process_time 測量時間：40.310300 秒

繼續抓

第26筆 萬華區東園街海鮮餐廳,總共有:102筆,已寫入完成
process_time 測量時間：43.409058 秒

注意!!!萬華區東園街:已全數寫入完成

萬華區東園街牛排需重新整理,搜尋逾時,本次刷新第1次
萬華區柳州街火鍋需重新整理,搜尋逾時,本次刷新第1次
第1筆 萬華區東園街燒肉,總共有:120筆,已寫入完成
process_time 測量時間：68.946130 秒

繼續抓

第2筆 萬華區東園街甜點店,總共有:120筆,已寫入完成
process_time 測量時間：77.567736 秒

繼續抓

第3筆 萬華區柳州街拉麵,總共有:120筆,已寫入完成
process_time 測量時間：41.204618 秒

繼續抓

萬華區柳州街美式需重新整理,搜尋逾時,本次刷新第1次
第4筆 萬華區柳州街日式,總共有:120筆,已寫入完成
process_time 測量時間：40.718268 秒

繼續抓

第5筆 萬華區柳州街火鍋,總共有:122筆,已寫入完成
process_time 測量時間：72.492286 秒

繼續抓

萬華區柳州街義式需重新整理,搜尋逾時,本次刷新第1次
萬華區柳州街法式需重新整理,搜尋逾時,本次刷新第1次
萬華區柳州街地中海料理需重新整理,搜尋逾時,本次刷新第1次
萬華區柳州街韓式需重新整理,搜尋逾時,本次刷新第1次
第6筆 萬華區柳州街台式,總共有:100筆,已寫入完成
process_time 測量時間：35.692567 秒

繼續抓

萬華區柳州街德式需重新整理,搜尋逾時,本次刷新第1次
第7筆 萬華區柳州街美式,總共有:120筆,已寫入完成
process_time 測量時間：85.417953 秒

繼

...

第23筆 萬華區環河南路２段牛排,總共有:71筆,已寫入完成
process_time 測量時間：56.000561 秒

萬華區環河南路３段日式需重新整理,搜尋逾時,本次刷新第1次
繼續抓

繼續抓

第24筆 萬華區環河南路３段義式,總共有:120筆,已寫入完成
process_time 測量時間：40.826203 秒

繼續抓

第25筆 萬華區環河南路３段法式,總共有:120筆,已寫入完成
process_time 測量時間：42.774419 秒

繼續抓

萬華區環河南路３段德式需重新整理,搜尋逾時,本次刷新第1次
第26筆 萬華區環河南路３段中式,總共有:120筆,已寫入完成
process_time 測量時間：42.196171 秒

注意!!!萬華區環河南路:已全數寫入完成

第1筆 萬華區環河南路３段韓式,總共有:120筆,已寫入完成
process_time 測量時間：45.162633 秒

繼續抓

萬華區環河南路３段越式需重新整理,搜尋逾時,本次刷新第1次
萬華區環河南路３段印度料理需重新整理,搜尋逾時,本次刷新第1次
萬華區環河南路３段餐酒館需重新整理,搜尋逾時,本次刷新第1次
萬華區環河南路３段咖啡廳需重新整理,搜尋逾時,本次刷新第1次
萬華區環河南路３段鐵板燒需重新整理,搜尋逾時,本次刷新第1次
第2筆 萬華區環河南路３段印度料理,總共有:81筆,已寫入完成
process_time 測量時間：78.891562 秒

萬華區環河南路３段熱炒店需重新整理,搜尋逾時,本次刷新第1次
繼續抓

第3筆 萬華區環河南路３段越式,總共有:120筆,已寫入完成
process_time 測量時間：89.297952 秒

繼續抓

萬華區興寧街火鍋需重新整理,搜尋逾時,本次刷新第1次
第4筆 萬華區興寧街拉麵,總共有:120筆,已寫入完成
process_time 測量時間：40.477684 秒

繼續抓

第5筆 萬華區興寧街日式,總共有:120筆,已寫入完成
process_time 測量時間：40.578133 秒

第6筆 萬華區興寧街美式,總共有:120筆,已寫入完成
process_time 測量時間：40.744266 秒

繼續抓

繼續抓

第7筆 萬華區環河南路３段鐵板燒,總共有:120筆,已寫入完成
process_

第24筆 萬華區華西街熱炒店,總共有:4筆,已寫入完成
process_time 測量時間：3.180793 秒

繼續抓

第25筆 萬華區華西街鐵板燒,總共有:120筆,已寫入完成
process_time 測量時間：42.269081 秒

繼續抓

萬華區華西街韓式需重新整理,搜尋逾時,本次刷新第4次
第26筆 萬華區華西街甜點店,總共有:2筆,已寫入完成
process_time 測量時間：3.234795 秒

注意!!!萬華區華西街:已全數寫入完成

第1筆 萬華區華西街餐酒館,總共有:120筆,已寫入完成
process_time 測量時間：63.911557 秒

繼續抓

第2筆 萬華區華西街咖啡廳,總共有:120筆,已寫入完成
process_time 測量時間：40.574970 秒

繼續抓

萬華區華西街韓式需重新整理,搜尋逾時,本次刷新第5次
萬華區華西街早午餐需重新整理,搜尋逾時,本次刷新第1次
第3筆 萬華區華西街海鮮餐廳,總共有:41筆,已寫入完成
process_time 測量時間：16.262225 秒

繼續抓

第4筆 萬華區華西街燒肉,總共有:120筆,已寫入完成
process_time 測量時間：40.758537 秒

繼續抓

第5筆 萬華區華西街牛排,總共有:120筆,已寫入完成
process_time 測量時間：40.853986 秒

繼續抓

第6筆 萬華區華西街早午餐,總共有:120筆,已寫入完成
process_time 測量時間：85.461421 秒

第7筆 萬華區華西街韓式,總共有:120筆,已寫入完成
process_time 測量時間：221.555323 秒

繼續抓

萬華區萬大路拉麵需重新整理,搜尋逾時,本次刷新第1次
繼續抓

第8筆 萬華區萬大路日式,總共有:28筆,已寫入完成
process_time 測量時間：11.257844 秒

繼續抓

第9筆 萬華區萬大路火鍋,總共有:120筆,已寫入完成
process_time 測量時間：40.528016 秒

繼續抓

第10筆 萬華區萬大路台式,總共有:49筆,已寫入完成
process_time 測量時間：19.452433 秒

繼續抓

萬華區萬大路中式需重新整理,搜尋逾時,本次刷新第1次
第11筆 萬華區萬大路拉

繼續抓

第26筆 萬華區西寧南路義式,總共有:120筆,已寫入完成
process_time 測量時間：42.786869 秒

注意!!!萬華區西寧南路:已全數寫入完成

第1筆 萬華區西寧南路法式,總共有:120筆,已寫入完成
process_time 測量時間：40.849589 秒

第2筆 萬華區西寧南路中式,總共有:120筆,已寫入完成
process_time 測量時間：40.549389 秒

繼續抓

萬華區西寧南路韓式需重新整理,搜尋逾時,本次刷新第1次
繼續抓

第3筆 萬華區西寧南路港式,總共有:14筆,已寫入完成
process_time 測量時間：6.637228 秒

繼續抓

第4筆 萬華區西寧南路德式,總共有:120筆,已寫入完成
process_time 測量時間：40.684513 秒

萬華區西寧南路越式需重新整理,搜尋逾時,本次刷新第1次
繼續抓

第5筆 萬華區西寧南路地中海料理,總共有:110筆,已寫入完成
process_time 測量時間：39.868772 秒

繼續抓

第6筆 萬華區西寧南路鐵板燒,總共有:4筆,已寫入完成
process_time 測量時間：3.198996 秒

繼續抓

萬華區西寧南路泰式需重新整理,搜尋逾時,本次刷新第1次
第7筆 萬華區西寧南路南洋,總共有:120筆,已寫入完成
process_time 測量時間：40.713072 秒

繼續抓

萬華區西寧南路素食需重新整理,搜尋逾時,本次刷新第1次
第8筆 萬華區西寧南路越式,總共有:120筆,已寫入完成
process_time 測量時間：71.126922 秒

繼續抓

第9筆 萬華區西寧南路泰式,總共有:120筆,已寫入完成
process_time 測量時間：78.159631 秒

繼續抓

萬華區西寧南路餐酒館需重新整理,搜尋逾時,本次刷新第1次
萬華區西寧南路咖啡廳需重新整理,搜尋逾時,本次刷新第1次
萬華區西寧南路素食需重新整理,搜尋逾時,本次刷新第2次
第10筆 萬華區西寧南路燒肉,總共有:60筆,已寫入完成
process_time 測量時間：21.165938 秒

繼續抓

萬華區西寧南路甜點店需重新整理,搜尋逾時,本次刷新第1次
第11筆 萬華區西寧南路早午餐,總共有:120筆,已寫入完成
proce

繼續抓

萬華區長沙街２段咖啡廳需重新整理,搜尋逾時,本次刷新第1次
第52筆 萬華區長沙街２段海鮮餐廳,總共有:20筆,已寫入完成
process_time 測量時間：8.368886 秒

注意!!!萬華區長沙街:已全數寫入完成

第1筆 萬華區長沙街２段早午餐,總共有:105筆,已寫入完成
process_time 測量時間：36.018646 秒

繼續抓

萬華區長沙街２段素食需重新整理,搜尋逾時,本次刷新第2次
萬華區長沙街２段燒肉需重新整理,搜尋逾時,本次刷新第1次
萬華區長沙街２段牛排需重新整理,搜尋逾時,本次刷新第1次
萬華區長泰街拉麵需重新整理,搜尋逾時,本次刷新第1次
萬華區長泰街火鍋需重新整理,搜尋逾時,本次刷新第1次
第2筆 萬華區長沙街２段咖啡廳,總共有:101筆,已寫入完成
process_time 測量時間：75.874593 秒

繼續抓

萬華區長沙街２段燒肉需重新整理,搜尋逾時,本次刷新第2次
萬華區長泰街美式需重新整理,搜尋逾時,本次刷新第1次
萬華區長泰街日式需重新整理,搜尋逾時,本次刷新第1次
萬華區長泰街義式需重新整理,搜尋逾時,本次刷新第1次
萬華區長沙街２段牛排需重新整理,搜尋逾時,本次刷新第2次
萬華區長沙街２段燒肉需重新整理,搜尋逾時,本次刷新第3次
萬華區長泰街美式需重新整理,搜尋逾時,本次刷新第2次
萬華區長泰街日式需重新整理,搜尋逾時,本次刷新第2次
第3筆 萬華區長沙街２段牛排,總共有:120筆,已寫入完成
process_time 測量時間：128.647130 秒

繼續抓

第4筆 萬華區長沙街２段燒肉,總共有:120筆,已寫入完成
process_time 測量時間：147.838621 秒

繼續抓

第5筆 萬華區長泰街法式,總共有:120筆,已寫入完成
process_time 測量時間：40.980139 秒

繼續抓

第6筆 萬華區長泰街美式,總共有:120筆,已寫入完成
process_time 測量時間：94.356398 秒

第7筆 萬華區長泰街日式,總共有:120筆,已寫入完成
process_time 測量時間：94.401669 秒

繼續抓

繼續抓

第8筆 萬華區長泰街台式,總共有:120筆,已寫入完成
process_time 測量時間：40.626362 秒



第21筆 萬華區雅江街甜點店,總共有:39筆,已寫入完成
process_time 測量時間：20.919390 秒

繼續抓

萬華區雅江街海鮮餐廳需重新整理,搜尋逾時,本次刷新第1次
萬華區雅江街早午餐需重新整理,搜尋逾時,本次刷新第1次
萬華區雅江街燒肉需重新整理,搜尋逾時,本次刷新第1次
第22筆 萬華區雅江街素食,總共有:120筆,已寫入完成
process_time 測量時間：84.596025 秒

繼續抓

第23筆 萬華區雙和街火鍋,總共有:9筆,已寫入完成
process_time 測量時間：4.821195 秒

繼續抓

萬華區雅江街牛排需重新整理,搜尋逾時,本次刷新第1次
萬華區雅江街海鮮餐廳需重新整理,搜尋逾時,本次刷新第2次
萬華區雅江街燒肉需重新整理,搜尋逾時,本次刷新第2次
第24筆 萬華區雅江街早午餐,總共有:189筆,已寫入完成
process_time 測量時間：78.142752 秒

繼續抓

第25筆 萬華區雅江街海鮮餐廳,總共有:71筆,已寫入完成
process_time 測量時間：89.930902 秒

繼續抓

第26筆 萬華區雅江街牛排,總共有:120筆,已寫入完成
process_time 測量時間：72.770489 秒

注意!!!萬華區雅江街:已全數寫入完成

第1筆 萬華區雅江街燒肉,總共有:120筆,已寫入完成
process_time 測量時間：110.878385 秒

繼續抓

第2筆 萬華區雙和街拉麵,總共有:117筆,已寫入完成
process_time 測量時間：42.602032 秒

繼續抓

第3筆 萬華區雙和街美式,總共有:120筆,已寫入完成
process_time 測量時間：40.701510 秒

繼續抓

第4筆 萬華區雙和街中式,總共有:58筆,已寫入完成
process_time 測量時間：21.078465 秒

繼續抓

萬華區雙和街義式需重新整理,搜尋逾時,本次刷新第1次
第5筆 萬華區雙和街法式,總共有:120筆,已寫入完成
process_time 測量時間：40.687984 秒

繼續抓

第6筆 萬華區雙和街台式,總共有:101筆,已寫入完成
process_time 測量時間：36.218913 秒

第7筆 萬華區雙和街德式,總共有:36筆,已

In [ ]:
# import re

# global index 
# index= 26
# link = "北投區一德街火鍋"

# if index%26 == 0:
#         regex01 = r'.+(街|路)'
#         string01 = link
#         match01 = re.search(regex01, string01)
#         print(f'注意!!!,{match01[0]}:已全數寫入完成\n')
#         index = 0
# else:
#     print("繼續抓\n")